# Transformers - Speech commands classification with Transformers

In [1]:
# Install dependencies
! pip install numpy
! pip install matplotlib
! pip install scikit-learn

# ! pip install setuptools
! pip install tensorflow --upgrade
! pip install keras --upgrade

  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl (15.8 MB)
  Using cached matplotlib-3.8.4-cp311-cp311-win_amd64.whl.metadata (5.9 kB)
  Using cached contourpy-1.2.1-cp311-cp311-win_amd64.whl.metadata (5.8 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.51.0-cp311-cp311-win_amd64.whl.metadata (162 kB)
  Using cached kiwisolver-1.4.5-cp311-cp311-win_amd64.whl.metadata (6.5 kB)
  Using cached pillow-10.3.0-cp311-cp311-win_amd64.whl.metadata (9.4 kB)
  Using cached pyparsing-3.1.2-py3-none-any.whl.metadata (5.1 kB)
Using cached matplotlib-3.8.4-cp311-cp311-win_amd64.whl (7.7 MB)
Using cached contourpy-1.2.1-cp311-cp311-win_amd64.whl (188 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
Using cached fonttools-4.51.0-cp311-cp311-win_amd64.whl (2.2 MB)
Using cached kiwisolver-1.4.5-cp311-cp311-win_amd64.whl (56 kB)
Using cached pillow-10.3.0-cp311-cp311-win_amd64.w

In [5]:
# Please read the guidelines on https://www.kaggle.com/docs/api#authentication
# to get an idea where to put your kaggle API token to be able to download the dataset

! pip install kaggle
! kaggle competitions download -c "tensorflow-speech-recognition-challenge"

! tar -xf tensorflow-speech-recognition-challenge.zip -C ../data/challenge
! del tensorflow-speech-recognition-challenge.zip

# To extract the dataset, we need to have 7-zip (https://www.7-zip.org/) installed and added to our $PATH
! 7z x -o..\data\challenge ..\data\challenge\test.7z
! 7z x -o..\data\challenge ..\data\challenge\train.7z



  0%|          | 0.00/3.50G [00:00<?, ?B/s]
  0%|          | 1.00M/3.50G [00:00<39:21, 1.59MB/s]
  0%|          | 2.00M/3.50G [00:00<20:06, 3.12MB/s]
  0%|          | 4.00M/3.50G [00:00<09:25, 6.65MB/s]
  0%|          | 6.00M/3.50G [00:00<06:31, 9.60MB/s]
  0%|          | 8.00M/3.50G [00:01<05:35, 11.2MB/s]
  0%|          | 10.0M/3.50G [00:01<05:00, 12.5MB/s]
  0%|          | 12.0M/3.50G [00:01<04:38, 13.5MB/s]
  0%|          | 14.0M/3.50G [00:01<04:25, 14.1MB/s]
  0%|          | 16.0M/3.50G [00:01<04:18, 14.5MB/s]
  1%|          | 18.0M/3.50G [00:01<04:00, 15.5MB/s]
  1%|          | 21.0M/3.50G [00:01<03:35, 17.3MB/s]
  1%|          | 23.0M/3.50G [00:02<03:30, 17.7MB/s]
  1%|          | 25.0M/3.50G [00:02<03:28, 17.9MB/s]
  1%|          | 27.0M/3.50G [00:02<03:28, 17.9MB/s]
  1%|          | 29.0M/3.50G [00:02<03:30, 17.8MB/s]
  1%|          | 31.0M/3.50G [00:02<03:35, 17.3MB/s]
  1%|          | 34.0M/3.50G [00:02<03:13, 19.3MB/s]
  1%|          | 36.0M/3.50G [00:02<03:22, 18.4MB/s]
 


7-Zip 23.01 (x64) : Copyright (c) 1999-2023 Igor Pavlov : 2023-06-20

Scanning the drive for archives:
1 file, 2640679130 bytes (2519 MiB)

Extracting archive: ..\data\challenge\test.7z
--
Path = ..\data\challenge\test.7z
Type = 7z
Physical Size = 2640679130
Headers Size = 1345163
Method = Delta LZMA2:24
Solid = +
Blocks = 3

Everything is Ok

Folders: 2
Files: 158538
Size:       5080191672
Compressed: 2640679130

7-Zip 23.01 (x64) : Copyright (c) 1999-2023 Igor Pavlov : 2023-06-20

Scanning the drive for archives:
1 file, 1121103842 bytes (1070 MiB)

Extracting archive: ..\data\challenge\train.7z
--
Path = ..\data\challenge\train.7z
Type = 7z
Physical Size = 1121103842
Headers Size = 389133
Method = Delta LZMA2:24
Solid = +
Blocks = 2

Everything is Ok

Folders: 33
Files: 64732
Size:       2055161498
Compressed: 1121103842
